In [14]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, sigmay
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp

N = 16
hi = nk.hilbert.Spin(s=1 / 2, N=N)
theta1 = 0.25+0.0j
theta2 = 0.25+0.0j
H = 2*sum([sigmax(hi, i)*sigmax(hi, i+1)+sigmay(hi, i)*sigmay(hi, i+1) for i in range(1, int(N/2))])
H += 2*sum([sigmam(hi, i)*sigmap(hi, (i+1)%N)+sigmap(hi, i)*sigmam(hi, (i+1)%N) for i in range(int(N/2+1), N)])
H += 2*jnp.cos(theta1)*(sigmax(hi, 0)*sigmax(hi, 1)+sigmay(hi, 0)*sigmay(hi, 1))+jnp.sin(theta1)*(sigmax(hi, 0)*sigmay(hi, 1)-sigmay(hi, 0)*sigmax(hi, 1))
H += 2*jnp.cos(theta2)*(sigmax(hi, int(N/2))*sigmax(hi, int(N/2)+1)+sigmay(hi, int(N/2))*sigmay(hi, int(N/2)+1))+jnp.sin(theta2)*(sigmax(hi, int(N/2))*sigmay(hi, int(N/2)+1)-sigmay(hi, int(N/2))*sigmax(hi, int(N/2)+1))
H += sum([sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])
sp_h=H.to_sparse()
eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("eigenvalues with scipy sparse:", eig_vals)

E_gs = eig_vals[0]

eigenvalues with scipy sparse: [-39.68304359 -38.43108525]


In [1]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, sigmay
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp


In [33]:
import netket as nk

# Define the Hilbert space, typically for a spin system
hilbert_space = nk.hilbert.Spin(s=0.5, N=1)  # Example for a single spin-1/2 system

# Define the σ^x and σ^z operators
sigmax = nk.operator.spin.sigmax(hilbert=hilbert_space, site=0)
sigmaz = nk.operator.spin.sigmaz(hilbert=hilbert_space, site=0)

# Convert operators to dense matrix form to access elements
matrix_sigmax = (identity(hilbert_space)-sigmax).to_dense()
matrix_sigmaz = sigmaz.to_dense()
print(matrix_sigmax)

[[ 1. -1.]
 [-1.  1.]]


In [57]:
import jax
import netket as nk
import numpy as np
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, identity
import time
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp
import matplotlib.pyplot as plt
from twoD_tool import *
Lx = 4
Ly = 4
N = Lx * Ly
params = [ 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2, 3.6, 4.0, 4.4 ]
Omega = 1.0
Rb = 1.2**6
hi = nk.hilbert.Spin(s=1 / 2, N =  N)
for param in params:
    ni = []
    H = Omega/2*sum([sigmax(hi, y*Lx+x) for y in range (Ly) for x in range (Lx)]) #X
    H -= param/2*sum([(identity(hi)-sigmaz(hi, y*Lx+x)) for y in range(Ly) for x in range(Lx)])
    H += Omega*Rb/4*sum([((identity(hi)-sigmaz(hi, y1*Lx+x1))*(identity(hi)-sigmaz(hi, y1*Lx+x2)))/((x1-x2)**2)**3 \
                         for y1 in range(Ly) for x1 in range(Lx) for x2 in range(x1+1, Lx)])
    H += Omega*Rb/4*sum([((identity(hi)-sigmaz(hi, y1*Lx+x1))*(identity(hi)-sigmaz(hi, y2*Lx+x2)))/(((x1-x2)**2+(y1-y2)**2)**3) \
                         for y1 in range(Ly) for x1 in range(Lx) for y2 in range(y1+1, Ly) for x2 in range(Lx)])
    sp_h = H.to_sparse()
    eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
    
    stagger_H = sum([sigmaz(hi, y*Lx+x)*(-1)**(y*Lx+x) for y in range(Ly) for x in range (Lx)])
    for y in range(Ly):
        for x in range(Lx):
            h_ni = (identity(hi)-sigmaz(hi, y*Lx+x))/2
            ni.append(eig_vecs[:,0].conj() @ h_ni.to_sparse() @ eig_vecs[:,0])
    stagger_H_sp = stagger_H.to_sparse()
    stagger_mag0 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,0])
    stagger_mag1 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,1])
    stagger_mag2 = np.abs(eig_vecs[:,1].conj() @ stagger_H_sp @ eig_vecs[:,1])
    print(stagger_mag0)
    print(stagger_mag1)
    print(stagger_mag2)
    print(eig_vals[1]-eig_vals[0])
    print(ni)

3.3306690738754696e-16
1.6653345369377348e-15
1.4432899320127035e-15
0.5419453674179435
[0.2684839829406159, 0.2684839829406155, 0.26848398294061565, 0.2684839829406158]
2.7755575615628914e-16
4.163336342344337e-16
2.7755575615628914e-17
0.37447890904652437
[0.3309494450665177, 0.330949445066517, 0.3309494450665171, 0.3309494450665176]
5.551115123125783e-16
4.163336342344337e-16
9.71445146547012e-17
0.2549717315401123
[0.38775084146740735, 0.38775084146740907, 0.38775084146740896, 0.38775084146740757]
5.273559366969494e-16
2.220446049250313e-16
1.3877787807814457e-16
0.17974790143583252
[0.43016763359411636, 0.43016763359412713, 0.43016763359412724, 0.4301676335941162]
2.498001805406602e-16
1.3877787807814457e-16
4.163336342344337e-17
0.13593893222297426
[0.4581735136134824, 0.45817351361346104, 0.4581735136134612, 0.45817351361348235]
4.579669976578771e-16
1.5959455978986625e-16
2.0816681711721685e-17
0.11164983697851394
[0.476556094174727, 0.47655609417476336, 0.4765560941747633, 0.4

In [54]:
hi = nk.hilbert.Spin(s=1 / 2, N =  1)
H = identity(hi)-sigmaz(hi, 0)
H.to_sparse() @ [1, 0]

array([0., 0.])

(512, 1)